In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install --upgrade pip
%pip install boto3 --quiet
%pip install botocore --quiet
%pip install json --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [13]:
import base64
import boto3
import json
from IPython.display import Image
from json_eval import compare_json



In [52]:
from llm_util import generate_conversation

def get_ocr(test):
    model_id : str = test['model_id']
    input_image : str = test['input_image']
    instructions_filename : str = test['instructions_filename']
    temperature : float = test['temperature']
    
    inference_config = {"temperature": temperature}
    # print(test['instructions_filename'])
    # print(f"./prompts/{instructions_filename}")
    instructions = open(f"./prompts/{instructions_filename}", "r").read()
    system_prompts = [{"text": instructions}]

    bedrock_client = boto3.client('bedrock-runtime',region_name='us-east-1')
    response = generate_conversation(
        bedrock_client, model_id, system_prompts, inference_config, input_image)

    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")
    

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

    return output_message['content'][0]['text']

In [53]:
#model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
base_test = {
                'model_id' : "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
                'input_image' : "./images/pizza-1.png",
                'instructions_filename' : 'pizza_system_prompt1.txt',
                'temperature' : 0
        }
from copy import deepcopy
tests = [deepcopy(base_test) for i in range(5)]
tests[0]['temperature'] = 0
tests[1]['temperature'] = 1
tests[2]['temperature'] = 1
tests[3]['temperature'] = 1
tests[4]['temperature'] = 1
print(tests[0])


{'model_id': 'us.anthropic.claude-3-5-sonnet-20240620-v1:0', 'input_image': './images/pizza-1.png', 'instructions_filename': 'pizza_system_prompt1.txt', 'temperature': 0}


In [54]:
def get_outfile_path(idx, t):
        return f'./outputs/pizza_actual_' + t['model_id'] + f"_tempr({t['temperature']})_" + t['instructions_filename'].replace('.txt', '') + f'_{idx}' + '.json'

for idx,t in enumerate(tests):
        outfile_path = get_outfile_path(idx, t)
        output = get_ocr(t)
        print(f'Writing output to {outfile_path}')
        open(outfile_path, 'w').write(output)

#actual_json = json_eval.extract_json_from_string(output)

Generating message with model us.anthropic.claude-3-5-sonnet-20240620-v1:0
Role: assistant
Text: Here's the extracted menu information in the requested JSON format:

{
  "categories": [
    {
      "name": "תפריט",
      "dishes": [
        {
          "name": "פיצה אישית (S)",
          "price": 20
        },
        {
          "name": "פיצה משפחתית (L)",
          "price": 29
        },
        {
          "name": "פיצה ענקית (XL)",
          "price": 49
        },
        {
          "name": "משפחתית טבעונית",
          "price": 40
        },
        {
          "name": "משפחתית ללא גלוטן",
          "price": 45
        },
        {
          "name": "המלצת השף פיצה יווניתת",
          "price": 49,
          "description": "עגבניה,בצל,זיתים שחורים,בולגרית"
        }
      ],
      "extraInfo": [
        "100% צהובה עמק",
        "100% מוצרלה"
      ]
    },
    {
      "name": "סלטים",
      "dishes": [
        {
          "name": "סלט בולגרית",
          "price": 34,
          "de

In [57]:
for idx,t in enumerate(tests):
    expected_json = json.load(open('./outputs/pizza_expected.json'))
    actual_output = open(get_outfile_path(idx, t), "r").read()
    actual = json.load(json_eval.extract_json_from_string(actual_output))
    
    similarity_score, difference_report = compare_json(expected, actual)
    print(f"Test({idx}) - Similarity score: {similarity_score:.2f}%")
    print("\nDetailed Difference Report:")
    difference_report.print_report()

In [14]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = actual_json

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 81.66%

Detailed Difference Report:
Differences found:
  Path: categories[0].dishes[3].description
  Type: Missing Key
  Value 1: V
  Value 2: Not present

  Path: categories[0].dishes[5].name
  Type: String Difference
  Value 1: המלצת השף פיצה יוונית
  Value 2: המלצת השף פיצה יווניתת

  Path: categories[0].extraInfo[0]
  Type: String Difference
  Value 1: תוספת שימושים בקצוות הפיצה 2 ₪
  Value 2: 100% גבינה עמק

  Path: categories[1].extraInfo[0]
  Type: String Difference
  Value 1: כל הסלטים מגיעים עם שתי לחמניות לבחירה
  Value 2: כל הסלטים מוגשים עם פיתה לבנה

  Path: categories[3].name
  Type: String Difference
  Value 1: רטבים
  Value 2: תוספות

  Path: categories[3].dishes[1].name
  Type: String Difference
  Value 1: רוטב פיצה, מיני סטייק
  Value 2: רוטב פיצה, מיני טוסטר

  Path: categories[4].dishes[4].name
  Type: String Difference
  Value 1: לחם שום ענק (L)
  Value 2: לחם שום הבית (L)

  Path: categories[4].dishes[4].description
  Type: String Difference
  Va

In [ ]:
# Example usage
import json
json1 = json.loads('{"name": "John", "age": 30, "city": "New York", "hobbies": ["reading", "swimming"]}')
json2 = json.loads('{"name": "Jon", "age": 31, "city": "New York", "hobbies": ["reading", "running"]}')

from json_eval import compare_json

similarity_score, difference_report = compare_json(json1, json2)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 77.76%

Detailed Difference Report:
Differences found:
  Path: age
  Type: Value Difference
  Value 1: 30
  Value 2: 31

  Path: name
  Type: String Difference
  Value 1: John
  Value 2: Jon



In [ ]:
# load json from outputs/groundtruth1.json
expected = json.load(open('./outputs/pizza_expected.json'))
actual = json.load(open('./outputs/pizza_actual.json'))
actual = json.load(open(output_filename_fullpath))

similarity_score, difference_report = compare_json(expected, actual)
print(f"Similarity score: {similarity_score:.2f}%")
print("\nDetailed Difference Report:")
difference_report.print_report()

Similarity score: 35.73%

Detailed Difference Report:
Differences found:
  Path: categories[0].extraInfo[0]
  Type: String Difference
  Value 1: תוספת שימושים בקצוות הפיצה 2 ₪
  Value 2: תוספת ביצה קשה לסלט 3 ש"ח

  Path: categories[0].dishes[0].price
  Type: Value Difference
  Value 1: 20
  Value 2: 34

  Path: categories[0].dishes[0].name
  Type: String Difference
  Value 1: פיצה אישית (S)
  Value 2: סלט טורקי

  Path: categories[0].dishes[0].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וזיתים שרויים

  Path: categories[0].dishes[1].price
  Type: Value Difference
  Value 1: 29
  Value 2: 34

  Path: categories[0].dishes[1].name
  Type: String Difference
  Value 1: פיצה משפחתית (L)
  Value 2: סלט בגרוזית

  Path: categories[0].dishes[1].description
  Type: Missing Key
  Value 1: Not present
  Value 2: חסה, מלפפון, עגבניה שרי, חצילים וביצים שרויים

  Path: categories[0].dishes[2].price
  Type: Value Difference
  Value 1: 49
  Value 2